# Sample Datasets

> **No sample data required!** This framework works directly with your own CSV, Parquet, or Delta files. The datasets below are internal examples for learning - skip to **01_data_discovery.ipynb** if you have your own data.

**Purpose:** Verify your environment and optionally download or generate sample datasets.

**What you'll do:**
- Verify your Python environment
- (Optional) Set up Kaggle API credentials
- (Optional) Download sample churn datasets
- (Optional) Generate synthetic multi-dataset bundles

---

## -1.6 Verify Environment

First, let's make sure the customer_retention package is installed.

In [ ]:
# @cr:code name='init_progress' id=e4b7f385
from customer_retention.analysis.notebook_progress import track_and_export_previous
from customer_retention.core.compat import native_pd

track_and_export_previous("00_start_here.ipynb")

try:
    import customer_retention
    from customer_retention.core.config.experiments import (
        EXPERIMENTS_DIR,
        FINDINGS_DIR,
        OUTPUT_DIR,
        setup_experiments_structure,
    )
    print("customer_retention is installed")
except ImportError:
    print("customer_retention not found. Install with:")
    print("  uv sync")
    print("  # or: pip install -e .")

## -1.5 Available Datasets

This framework includes several internal datasets for testing and learning. **You do not need any of these to use the framework with your own data.**

### Entity-Level Datasets (one row per customer)
Use these with the standard exploration flow (notebooks 02, 03, 04).

| Dataset | Status | Description |
|---------|--------|-------------|
| `customer_retention_retail.csv` | Included | Retail customer retention (~31K rows) |
| `bank_customer_churn.csv` | Download | Bank customer churn (~10K rows) |
| `netflix_customer_churn.csv` | Download | Netflix subscription churn (~10K rows) |

### Event-Level Datasets (multiple rows per customer)
Use these with the Event Bronze Track (notebooks 01a, 01b, 01c, 01d).

| Dataset | Status | Description |
|---------|--------|-------------|
| `customer_transactions.csv` | Included | Transaction events (~5K rows) |
| `customer_emails.csv` | Included | Email engagement events (large) |

### Multi-Dataset Bundle: EDI Ticketing 3-Set (self-generated)
Three related datasets designed to work together for multi-dataset workflows. Generated locally via the script in section -1.1 — no download required.

| Dataset | Granularity | Description |
|---------|-------------|-------------|
| `3set_customer_profiles.csv` | Entity-level | Customer demographics + churn label (target holder) |
| `3set_edi_transactions.csv` | Event-level | EDI transaction events per customer |
| `3set_support_tickets.csv` | Event-level | Support ticket events per customer |

All three share `customer_id` as the join key. The profiles dataset holds the binary `churned` target; the other two provide behavioral event histories. Size and churn rate are configurable at generation time (default: 1,000 customers, 13% churn, 4 years of history).

In [ ]:
# @cr:code name='define_datasets' id=462ad2f5
from pathlib import Path

FIXTURES_DIR = Path("../tests/fixtures")

# Entity-level datasets
entity_datasets = {
    "customer_retention_retail.csv": "Included",
    "bank_customer_churn.csv": "Download from Kaggle",
    "netflix_customer_churn.csv": "Download from Kaggle",
}

# Event-level datasets (internal)
event_datasets = {
    "customer_transactions.csv": "Included",
    "customer_emails.csv": "Included",
}

print("Entity-Level Datasets:")
print("-" * 50)
for name, source in entity_datasets.items():
    path = FIXTURES_DIR / name
    if path.exists():
        size_mb = path.stat().st_size / (1024 * 1024)
        print(f"  [x] {name} ({size_mb:.1f} MB)")
    else:
        print(f"  [ ] {name} - {source}")

print("\nEvent-Level Datasets:")
print("-" * 50)
for name, source in event_datasets.items():
    path = FIXTURES_DIR / name
    if path.exists():
        size_mb = path.stat().st_size / (1024 * 1024)
        print(f"  [x] {name} ({size_mb:.1f} MB)")
    else:
        print(f"  [ ] {name} - {source}")

## -1.4 Kaggle API Setup

To download datasets from Kaggle, you need to set up API credentials:

1. Create a Kaggle account at https://www.kaggle.com
2. Go to **Account Settings** → **API** → **Create New Token**
3. This downloads `kaggle.json` - move it to `~/.kaggle/kaggle.json`
4. Set permissions: `chmod 600 ~/.kaggle/kaggle.json`

In [ ]:
# @cr:code name='check_kaggle' id=7471c8de
# Check if Kaggle credentials exist
kaggle_config = Path.home() / ".kaggle" / "kaggle.json"

if kaggle_config.exists():
    print(f"Kaggle credentials found at {kaggle_config}")
else:
    print("Kaggle credentials not found.")
    print("\nTo set up:")
    print("1. Go to https://www.kaggle.com/settings")
    print("2. Scroll to 'API' section and click 'Create New Token'")
    print(f"3. Move downloaded file to {kaggle_config}")
    print(f"4. Run: chmod 600 {kaggle_config}")

## -1.3 Download Kaggle Datasets

Run the cells below to download each dataset. You only need to do this once.

### Bank Customer Churn Dataset
Source: https://www.kaggle.com/datasets/gauravtopre/bank-customer-churn-dataset

In [ ]:
# @cr:code name='download_bank_churn' id=cce83896
# Download Bank Customer Churn dataset
import shutil
import subprocess

FIXTURES_DIR.mkdir(parents=True, exist_ok=True)
bank_churn_path = FIXTURES_DIR / "bank_customer_churn.csv"

if bank_churn_path.exists():
    print(f"Already exists: {bank_churn_path}")
else:
    print("Downloading Bank Customer Churn dataset...")
    try:
        subprocess.run([
            "kaggle", "datasets", "download", "-d", "gauravtopre/bank-customer-churn-dataset",
            "-p", str(FIXTURES_DIR), "--unzip"
        ], check=True)
        # Rename to consistent name
        downloaded = FIXTURES_DIR / "Bank_Churn.csv"
        if downloaded.exists():
            shutil.move(downloaded, bank_churn_path)
        print(f"Downloaded to: {bank_churn_path}")
    except FileNotFoundError:
        print("Error: kaggle CLI not found. Install with: pip install kaggle")
    except subprocess.CalledProcessError as e:
        print(f"Error downloading: {e}")

### Netflix Customer Churn Dataset
Source: https://www.kaggle.com/datasets/vasifasad/netflix-customer-churn-prediction

In [ ]:
# @cr:code name='download_netflix' id=0ac45247
# Download Netflix Customer Churn dataset
netflix_churn_path = FIXTURES_DIR / "netflix_customer_churn.csv"

if netflix_churn_path.exists():
    print(f"Already exists: {netflix_churn_path}")
else:
    print("Downloading Netflix Customer Churn dataset...")
    try:
        subprocess.run([
            "kaggle", "datasets", "download", "-d", "vasifasad/netflix-customer-churn-prediction",
            "-p", str(FIXTURES_DIR), "--unzip"
        ], check=True)
        print(f"Downloaded to: {netflix_churn_path}")
    except FileNotFoundError:
        print("Error: kaggle CLI not found. Install with: pip install kaggle")
    except subprocess.CalledProcessError as e:
        print(f"Error downloading: {e}")

## -1.2 Verify Downloads

In [ ]:
# @cr:code name='dataset_summary' id=49d3c40d

all_datasets = {**entity_datasets, **event_datasets}

print("Dataset Summary:")
print("=" * 60)

for name in all_datasets.keys():
    path = FIXTURES_DIR / name
    if path.exists():
        df = native_pd.read_csv(path)
        print(f"\n{name}:")
        print(f"  Rows: {len(df):,}")
        print(f"  Columns: {len(df.columns)}")
        print(f"  Columns: {', '.join(df.columns[:5])}{'...' if len(df.columns) > 5 else ''}")
    else:
        print(f"\n{name}: Not downloaded")

## -1.1 Generate EDI Ticketing 3-Set (Optional)

Generate a synthetic multi-dataset bundle: **customer profiles** (entity-level, holds the churn label), **EDI transactions** (event-level), and **support tickets** (event-level). These three datasets are designed to work together for the multi-dataset workflow.

Set `GENERATE_EDI_DATASET = True` to run. Adjust `N_CUSTOMERS` and `CHURN_RATE` to control size and label balance.

In [ ]:
# @cr:code name='generate_edi_tickets' id=01ae79a0
# =============================================================================
# EDI TICKETING 3-SET GENERATOR — disabled by default
# =============================================================================
GENERATE_EDI_DATASET = False
N_CUSTOMERS = 1_000
CHURN_RATE = 0.13
N_YEARS = 4
SEED = 42

if GENERATE_EDI_DATASET:
    import sys

    sys.path.insert(0, str(Path("../scripts/data").resolve()))
    from generate_edi_ticketing_dataset import generate

    df_profiles, df_transactions, df_tickets = generate(
        output_dir=FIXTURES_DIR,
        n_customers=N_CUSTOMERS,
        churn_rate=CHURN_RATE,
        seed=SEED,
        n_years=N_YEARS,
    )
    print(f"\nGenerated 3-set: {N_CUSTOMERS:,} customers, {CHURN_RATE:.0%} churn, {N_YEARS}y history")
    print(f"  3set_customer_profiles.csv  — {len(df_profiles):,} rows")
    print(f"  3set_edi_transactions.csv   — {len(df_transactions):,} rows")
    print(f"  3set_support_tickets.csv    — {len(df_tickets):,} rows")
else:
    print("EDI 3-set generation skipped. Set GENERATE_EDI_DATASET = True to generate.")

## -1.0 Next Steps

You're ready to start exploring! Continue to **00_start_here.ipynb**.